# Integer Linear Programming for the CVRP
In our formulation, we have a fleet of $F$ identical vehicles each with some capacity $Q \in \mathbb{Z}$, and a graph $(V,E)$ of $N$ nodes each having placed an order of size $C_i$ with distances $d_{ij}$ between nodes $i$ and $j$. The 0th node is the depot, where all routes must start and end, and the problem is to find routes of the vehicles such that all demands are met, and such that some objective 'cost' function is minimised. 

Let $x_{ijk}$ be binary variables where $x_{ijk} = \begin{cases} 1 & \text{ if vehicle } k \text{ travels along the edge }E_{ij} \\ 0 & \text{if not}\end{cases}$, then the cost funciton is taken to be 
$$ \sum_{i,j,k}d_{ij}x_{ijk}.
$$
We now outline the constraints that are imposed to model the problem;

- Each vehicle must exit a node that it enters: For all vehicles $k$, and all nodes $j$ we have 
$$ \sum_{i} x_{ijk} - \sum_{i} x_{jik} =  0$$
- We can then impose that all non-depot nodes $j$ must be entered exactly once by some vehicle (note: in some formulations of this problem this can be relaxed, ie. we can skip customers if getting to them is a major hassle, we shall not neglect our customers in such a way. One could spend a lifetime going through every different formulation of VRP - hence our methods are focussed on only one instance!):
$$ \sum_{i,k} x_{ijk} =1, \text{ where } j>0$$
-  Moreover, all vehicle's routes should leave (equivalently, by the first condition, return to) the depot:
$$ \sum_{j} x_{0jk } =1$$
- And finally, we add in our capacity constraints (note that setting $C_0 =0$ means we dont have to change range we sum over):
$$ \sum_{i,j} C_jx_{ijk} $$

Coolio, however there is a minor (major?!) problem in that this formulation allows for 'subtours' to arise. The are two standard ways to try and eliminate subtours, one due to Miller-Tucker-Zemlin (MTZ), and the other due to Dantzig-Fulkerson-Johnson (DFJ). 

The fist method (MTZ) introduces new variables $u_i$ to fix the problem - essentially giving each node a 'running count' if it's been visited, which has to be increasing. The problem is very nicely outlined here: https://how-to.aimms.com/Articles/332/332-Miller-Tucker-Zemlin-formulation.html (with nice pictures also.)

Once we've added all of our extra variables $u_i$, the following constraint can be used to ensure that no subtours arise
$$ \begin{align} u_j - u_i \geq C_i - Q(1-x_{ijk}) &\text{ for distinct } i,j \in V \setminus \{1\} \\ C_i \leq u_i \leq Q &\text{ for all } i \in V \setminus \{1\}\end{align} $$ 

Explanation?!?!?

The second method (DFJ) introduces *loads* of contraints (one for each subset of the vertices) to ensure that this subset doesn't have a closed loop. For example for a subset $S \subset V$, one can impose that the number of edges travelled between nodes must be strictly less than the number of nodes in that subset. In fact, due to our other constraints above this can be written as 

$$ \sum_{i,j \in S, k } x_{ijk} = |S|-1 $$

We'll use MTZ - perhaps shall also implement DFJ.

Anyhoo, since ILP is an NP-hard problem, solving CVRP this way is not particularly scalable, so we dont want to invest too much time here. Rather than working from the ground up, we shall use the PuLP library for optimistaion. 


In background research - for example the papaer  - exact methods (while interesting) are generally only useful in cases up to round 60 vertices on our graph. We expect to have more here, so shall turn to heuristic approaches.


In [1]:
import pulp
import numpy as np




Okay, what methods do we have for solving ILP? We could relax to a LP program - then use simplex. 